In [1]:
# 기존 knowledge retrieval 대체

# Supported values are: 'code_interpreter', 'function', and 'file_search'."라는 오류가 발생합니다. 
# retrieval를 file_search로 변경한 예제입니다.
import os
import keyring
from openai import OpenAI

client = OpenAI(
    api_key=keyring.get_password('openai', 'key_for_mac')
)

In [2]:
import json
def show_json(obj):
    display(json.loads(obj.model_dump_json()))
    
assistant = client.beta.assistants.create(
    name='smartfarm',
    instructions='당신은 스마트팜에 대한 전문가입니다. pdf 파일에 기반하여 답변해주세요.',
    model='gpt-4o',
    tools=[{'type': 'file_search'}]
)
show_json(assistant)

{'id': 'asst_aihmGiR98hXUrwpvR6Hz586P',
 'created_at': 1722057719,
 'description': None,
 'instructions': '당신은 스마트팜에 대한 전문가입니다. pdf 파일에 기반하여 답변해주세요.',
 'metadata': {},
 'model': 'gpt-4o',
 'name': 'smartfarm',
 'object': 'assistant',
 'tools': [{'type': 'file_search', 'file_search': None}],
 'response_format': 'auto',
 'temperature': 1.0,
 'tool_resources': {'code_interpreter': None,
  'file_search': {'vector_store_ids': []}},
 'top_p': 1.0}

In [3]:
vector_store = client.beta.vector_stores.create(
    name="file_store"
)

# 업로드할 파일 PDF 준비
file_paths = ["data/smartfarm.pdf"]
file_streams = [open(path, 'rb') for path in file_paths]

# 파일을 업로드하고, vector store에 추가
file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
    vector_store_id=vector_store.id, files=file_streams,
)

# 파일 업로드 상태 확인
print(file_batch.status)
print(file_batch.file_counts)

show_json(vector_store)

completed
FileCounts(cancelled=0, completed=1, failed=0, in_progress=0, total=1)


{'id': 'vs_LFYzmUuQ7TY4kuxnaUnssuqE',
 'created_at': 1722058209,
 'file_counts': {'cancelled': 0,
  'completed': 0,
  'failed': 0,
  'in_progress': 0,
  'total': 0},
 'last_active_at': 1722058209,
 'metadata': {},
 'name': 'file_store',
 'object': 'vector_store',
 'status': 'completed',
 'usage_bytes': 0,
 'expires_after': None,
 'expires_at': None}

In [4]:
# 생성한 vector store를 참조할 수 있도록 업데이트
assistant = client.beta.assistants.update(
    assistant_id=assistant.id,
    tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}},
)
show_json(assistant)

{'id': 'asst_aihmGiR98hXUrwpvR6Hz586P',
 'created_at': 1722057719,
 'description': None,
 'instructions': '당신은 스마트팜에 대한 전문가입니다. pdf 파일에 기반하여 답변해주세요.',
 'metadata': {},
 'model': 'gpt-4o',
 'name': 'smartfarm',
 'object': 'assistant',
 'tools': [{'type': 'file_search', 'file_search': None}],
 'response_format': 'auto',
 'temperature': 1.0,
 'tool_resources': {'code_interpreter': None,
  'file_search': {'vector_store_ids': ['vs_LFYzmUuQ7TY4kuxnaUnssuqE']}},
 'top_p': 1.0}

In [5]:
# thread, message, run 생성 및 실행
thread = client.beta.threads.create()

message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role='user',
    content='스마트팜 발전 추진배경?',
)

run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id,
    assistant_id=assistant.id
)
show_json(run)

{'id': 'run_IzTlsYBkdX8Ch08sX9IdFz4s',
 'assistant_id': 'asst_aihmGiR98hXUrwpvR6Hz586P',
 'cancelled_at': None,
 'completed_at': 1722058529,
 'created_at': 1722058519,
 'expires_at': None,
 'failed_at': None,
 'incomplete_details': None,
 'instructions': '당신은 스마트팜에 대한 전문가입니다. pdf 파일에 기반하여 답변해주세요.',
 'last_error': None,
 'max_completion_tokens': None,
 'max_prompt_tokens': None,
 'metadata': {},
 'model': 'gpt-4o',
 'object': 'thread.run',
 'parallel_tool_calls': True,
 'required_action': None,
 'response_format': 'auto',
 'started_at': 1722058519,
 'status': 'completed',
 'thread_id': 'thread_Dqv52sxMLW2UFXFK16Otq1y6',
 'tool_choice': 'auto',
 'tools': [{'type': 'file_search', 'file_search': None}],
 'truncation_strategy': {'type': 'auto', 'last_messages': None},
 'usage': {'completion_tokens': 524,
  'prompt_tokens': 11549,
  'total_tokens': 12073},
 'temperature': 1.0,
 'top_p': 1.0,
 'tool_resources': {}}

In [6]:
# GPT-4o 답변 출력
if run.status == "completed":
    messages = client.beta.threads.messages.list(
        thread_id=thread.id
    )
    show_json(messages)
else:
    print(run.status)

{'data': [{'id': 'msg_bWpsQJ04J4h2JUZjiwvl2d5o',
   'assistant_id': 'asst_aihmGiR98hXUrwpvR6Hz586P',
   'attachments': [],
   'completed_at': None,
   'content': [{'text': {'annotations': [{'end_index': 256,
        'file_citation': {'file_id': 'file-qVfta4bTyylLZRufC9RlJtBX'},
        'start_index': 244,
        'text': '【4:0†source】',
        'type': 'file_citation'},
       {'end_index': 452,
        'file_citation': {'file_id': 'file-qVfta4bTyylLZRufC9RlJtBX'},
        'start_index': 440,
        'text': '【4:0†source】',
        'type': 'file_citation'},
       {'end_index': 635,
        'file_citation': {'file_id': 'file-qVfta4bTyylLZRufC9RlJtBX'},
        'start_index': 623,
        'text': '【4:0†source】',
        'type': 'file_citation'},
       {'end_index': 821,
        'file_citation': {'file_id': 'file-qVfta4bTyylLZRufC9RlJtBX'},
        'start_index': 809,
        'text': '【4:0†source】',
        'type': 'file_citation'}],
      'value': '스마트팜의 발전 추진 배경은 다음과 같습니다:\n\n1. **글로벌

In [10]:
# # old code for knowledge retreival

# from openai import OpenAI
# import keyring

# client = OpenAI(
#     api_key=keyring.get_password('openai', 'key_for_mac')
# )

# # file upload
# file = client.files.create(file=open('data/smartfarm.pdf', "rb"), purpose='assistants')

# # Assistant 생성
# assistant = client.beta.assistants.create(
#     instructions="You are a Q&A bot, and you have access to files to answer questions about the company.",
#     name='Q&A bot',
#     tools=[{'type': 'retrieval'}],      # deprecated
#     model='gpt-4o',
#     file_ids=[file.id],
# )

# # Thread 생성
# thread = client.beta.threads.create()

# # 생성한 thread에 Message 생성
# message = client.beta.threads.messages.create(
#     thread_id=thread.id,
#     role='user',
#     content='what is korean smart farm?',
# )

# # Assistant 실행
# run = client.beta.threads.runs.create(
#     thread_id=thread.id,
#     assistant_id=assistant.id
# )

# # Retrieve the run
# run = client.beta.threads.runs.retrieve(
#     thread_id=thread.id,
#     run_id=run.id
# )

# # 생성한 thread에 Message 생성
# messages = client.beta.threads.messages.list(
#     thread_id=thread.id,
# )

# for i, message in enumerate(reversed(messages.data), start=1):
#     print(f"\n# Message: {i}:")
#     for content in message.content:
#         print(content.text.value)
